In [80]:
# Mandatory cell for the rest of this assignment

%load_ext autoreload
%autoreload 2

from sys import path

path.append('../scripts')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
# Import des modules nécessaires
from node import BlockchainNode
from network import Node
from wallet import Wallet
from proof_of_stake import ProofOfStake
from smart_contract import SmartContractDefinition, SmartContractWritingOperation

# Création de plusieurs wallets pour les nœuds de la blockchain
walletNode1 = Wallet()
walletNode2 = Wallet()
walletNode3 = Wallet()

# Création d'un wallet utilisateur (ex: Alice)
walletAlice = Wallet()

# Initialisation du mécanisme de Proof of Stake avec le premier nœud
pos = ProofOfStake(walletNode1.publicKey)

#On reset le network
Node.reset_network()

#On connecte les 3 noeuds au network
node1 = BlockchainNode(walletNode1,pos)
node2 = BlockchainNode(walletNode2,pos)
node3 = BlockchainNode(walletNode3,pos)

assert len(Node.get_all_nodes())==3
"Success"

'Success'

In [84]:
# Exemple de source de code pour le smart contract
source_code = """
class MyNFTContract:
    def __init__(self, issuerPublicKey):
        self.issuerPublicKey = issuerPublicKey
        self.nfts = {}  # Dictionnaire des NFTs

    def mint(self, ownerPublicKey, nftId):
        if nftId not in self.nfts:
            self.nfts[nftId] = ownerPublicKey
            print(f"NFT {nftId} frappé par {ownerPublicKey}")
        else:
            raise ValueError(f"NFT {nftId} déjà existant")

    def transfer(self, senderPublicKey, receiverPublicKey, nftId):
        if nftId in self.nfts and self.nfts[nftId] == senderPublicKey:
            self.nfts[nftId] = receiverPublicKey
            print(f"NFT {nftId} transféré de {senderPublicKey} à {receiverPublicKey}")
        else:
            raise ValueError("NFT inexistant ou propriété invalide")
"""

# Création du Smart Contract (définition)
contract_definition = SmartContractDefinition(issuerPublicKey=walletAlice.publicKey, sourceCode=source_code, Ntoken=5)

# Signer le certificat
walletAlice.sign(contract_definition)

# Instanciation du Smart Contract
contract_instance = contract_definition.instantiate_contract()

# Ajouter le Smart Contract au premier nœud (node1)
node1.new_certificate(contract_definition) # Ajouter le smart contract au nœud

print(node1.blockchain.blockList)
print(node2.blockchain.blockList)
print(node3.blockchain.blockList)

Échec du forger du bloc. Diffusion du certificat : <smart_contract.SmartContractDefinition object at 0x0000027244D9A360>
Échec du forger du bloc. Diffusion du certificat : <smart_contract.SmartContractDefinition object at 0x0000027244D9A360>
Échec du forger du bloc. Diffusion du certificat : <smart_contract.SmartContractDefinition object at 0x0000027244D9A360>


In [85]:
# Frapper un NFT pour Alice via node1
contract_definition.mint(ownerPublicKey="Alice", nftId="NFT1")

NFT NFT1 frappée par Alice


In [86]:
print(contract_definition.owners)

{'NFT1': 'Alice'}


In [87]:
#Transferer un NFT de Alice à Edouard
contract_definition.transfer(senderPublicKey="Alice", receiverPublicKey="Edouard", nftId="NFT1")
print(contract_definition.owners)

NFT NFT1 transférée de Alice à Edouard
{'NFT1': 'Edouard'}
